Original Notebook:

https://www.kaggle.com/code/guanyuzhen/drw-crypto-ensemble

Gambit - giving up material for an advantage in time

In [1]:
import pandas as pd

def iBlend(path_to_ds, file_short_names, sls):

    def tida(sls):
        
        def read_subm(sls,i):
            tnm = sls["subm"][i]["name"]
            FiN = sls["path"] + tnm + ".csv"
            return pd.read_csv(FiN).rename(columns={'target':tnm, sls["target"]:tnm})
        
        dfs_subm = [read_subm(sls,i) for i in range(len(sls["subm"]))]
        df_subms = pd.merge(dfs_subm[0],  dfs_subm[1], on=['ID'])
        
        for i in range(2, len(sls["subm"])): 
            df_subms = pd.merge(df_subms, dfs_subm[i], on=['ID'])
            
        cols = [col for col in df_subms.columns if col != "ID"]
        short_name_cols = [c.replace(sls["prefix"], '') for c in cols]
        corrects = [wt for wt in sls["subwts"]]
        weights = [subm['weight'] for subm in sls["subm"]]
        
        def alls(x, cs=cols):
            tes = {c: x[c] for c in cs}.items()
            subms_sorted = [
              t[0].replace(sls["prefix"], '')
              for t in sorted(tes,key=lambda k:k[1],reverse=True if sls["sort"]=='desc' else False)]
            return subms_sorted
        
        def correct(x, cs=cols, w=weights, cw=corrects):
            ic = [x['alls'].index(c) for c in short_name_cols]
            cS = [x[cols[j]] * (w[j] + cw[ic[j]]) for j in range(len(cols))]
            return sum(cS)
        
        df_subms['alls']        = df_subms.apply(lambda x: alls   (x), axis=1)
        df_subms[sls["target"]] = df_subms.apply(lambda x: correct(x), axis=1)
        
        schema_rename = { old_nc:new_shnc for old_nc, new_shnc in zip(cols, short_name_cols) }
        
        df_subms = df_subms.rename(columns=schema_rename)
        df_subms = df_subms.rename(columns={sls["target"]:"ensemble"})
        
        df_subms.insert(loc=1, column=' _ ', value=['   '] * sls["q_rows"])
        
        df_subms[' _ '] = df_subms[' _ '].astype(str)
        pd.set_option('display.max_rows',100)
        pd.set_option('display.float_format', '{:.7f}'.format)
        vcols = ['ID'] + [' _ '] + short_name_cols + [' _ '] + ['alls'] + [' _ '] + ['ensemble']
        df_subms = df_subms[vcols]
        display(df_subms.head(7))
        pd.set_option('display.float_format', '{:.11f}'.format)
        df_subms = df_subms.rename(columns={"ensemble":sls["target"]})
        
        return df_subms
        

    sample_subm = pd.read_csv(path_to_ds + file_short_names[1] + ".csv")

    
    def ensemble_tida(sls,submission=sample_subm):   
        sls['sort'] = 'desc'
        dfs = tida(sls)
        dfD = dfs[['ID', sls['target']]]
        dfD.to_csv(f'tida_desc.csv', index=False)
        sls['sort'] = 'asc'
        dfs = tida(sls)
        dfA = dfs[['ID', sls['target']]]
        dfA.to_csv(f'tida_asc.csv',  index=False)
        target,d,a = sls['target'],sls['desc'],sls['asc']
        submission[target] = dfD[target] * d + a * dfA[target]
        return submission

    submission = ensemble_tida(sls)
    
    return submission
    
path_to_ds ='/kaggle/input/15-juli-2025-drw/submission '

file_short_names = ['0.83975','0.86767','0.89178']

params = {
      'path'  : path_to_ds,                                 
      'sort'  : "dynamic",
      'target': "prediction",
      'q_rows': 538_150,
      'prefix': "subm_",
      'desc'  : 0.30,
      'asc'   : 0.70,
      'subwts': [+0.40, -0.10, -0.30],                          # LB = 0.93980
      'subm'  : [
        { 'name':file_short_names[0],'weight':0.20, },
        { 'name':file_short_names[1],'weight':0.30, },
        { 'name':file_short_names[2],'weight':0.50, },
      ]
    }

path_to_ds ='/kaggle/input/15-juli-2025-drw/submission '

file_short_names = ['0.83975','0.86767','0.88377','0.89178','0.90038']

params = {
      'path'  : path_to_ds,                                 
      'sort'  : "dynamic",
      'target': "prediction",
      'q_rows': 538_150,
      'prefix': "subm_",
      'desc'  : 0.30,
      'asc'   : 0.70,
      'subwts': [+0.20, +0.10, -0.05,-0.10,-0.15],              # LB = ?
      'subm'  : [
         { 'name':file_short_names[0],'weight':0.20, },
         { 'name':file_short_names[1],'weight':0.20, },
         { 'name':file_short_names[2],'weight':0.21, },
         { 'name':file_short_names[3],'weight':0.22, },
         { 'name':file_short_names[4],'weight':0.23, },
      ]
    }

df = iBlend ( path_to_ds, file_short_names, params )

df.to_csv('submission.csv', index=False)

display(df)

,ID,_,0.83975,0.86767,0.88377,0.89178,0.90038,_,alls,_,ensemble
0,1,,0.0094010,-0.1066127,-0.1901365,-0.1910402,-0.1758105,,"[0.83975, 0.86767, 0.90038, 0.88377, 0.89178]",,-0.0941571
1,2,,0.2491083,0.4158519,0.2150628,0.1964704,0.2254042,,"[0.86767, 0.83975, 0.90038, 0.88377, 0.89178]",,0.3190558
2,3,,-1.5314554,-1.9806400,-1.0993711,-0.9708478,-0.9993967,,"[0.89178, 0.90038, 0.88377, 0.83975, 0.86767]",,-1.1656339
3,4,,-0.2361154,-0.2370307,-0.1544300,-0.1566623,-0.1548349,,"[0.88377, 0.90038, 0.89178, 0.83975, 0.86767]",,-0.1765075
4,5,,-0.1991619,0.2886727,0.1734419,0.1780911,0.1875781,,"[0.86767, 0.90038, 0.89178, 0.88377, 0.83975]",,0.2167659
5,6,,-0.1688776,-0.4720457,-0.7989751,-0.7657419,-0.7637376,,"[0.83975, 0.86767, 0.90038, 0.89178, 0.88377]",,-0.4864651
6,7,,-0.3883938,1.2381859,1.3843861,1.9755334,2.0334823,,"[0.90038, 0.89178, 0.88377, 0.86767, 0.83975]",,1.8324688


,ID,_,0.83975,0.86767,0.88377,0.89178,0.90038,_,alls,_,ensemble
0,1,,0.0094010,-0.1066127,-0.1901365,-0.1910402,-0.1758105,,"[0.89178, 0.88377, 0.90038, 0.86767, 0.83975]",,-0.1810163
1,2,,0.2491083,0.4158519,0.2150628,0.1964704,0.2254042,,"[0.89178, 0.88377, 0.90038, 0.83975, 0.86767]",,0.2354632
2,3,,-1.5314554,-1.9806400,-1.0993711,-0.9708478,-0.9993967,,"[0.86767, 0.83975, 0.88377, 0.90038, 0.89178]",,-1.6254729
3,4,,-0.2361154,-0.2370307,-0.1544300,-0.1566623,-0.1548349,,"[0.86767, 0.83975, 0.89178, 0.90038, 0.88377]",,-0.2216738
4,5,,-0.1991619,0.2886727,0.1734419,0.1780911,0.1875781,,"[0.83975, 0.88377, 0.89178, 0.90038, 0.86767]",,0.0431965
5,6,,-0.1688776,-0.4720457,-0.7989751,-0.7657419,-0.7637376,,"[0.88377, 0.89178, 0.90038, 0.86767, 0.83975]",,-0.7657384
6,7,,-0.3883938,1.2381859,1.3843861,1.9755334,2.0334823,,"[0.83975, 0.86767, 0.88377, 0.89178, 0.90038]",,0.8373426


,ID,prediction
0,1,-0.15495855126
1,2,0.26054099106
2,3,-1.48752122943
3,4,-0.20812392374
4,5,0.09526730365
...,...,...
538145,538146,-0.31601460746
538146,538147,0.14684046475
538147,538148,-0.94636983446
538148,538149,0.76784674350


Result

Score: 0.94857

Rank: 29 (2025-07-16-19:36, JST)

Runtime: 5min

Your Best Entry!
Your most recent submission scored 0.94857, which is an improvement of your previous score of 0.93980. Great job!

Moved up to rank 29 on #kaggle. I'm not addicted. I can quit when I want. https://kaggle.com/competitions/drw-crypto-market-prediction 